In [1]:
import OLS_regression_analysis_all_changes
import bbgclient
import datetime
import dfutils
import pandas as pd

In [2]:
metrics = ['P/E', "EV/EBITDA", "EV/Sales", 'DVD Yield', 'FCF Yield']
alpha_ticker = 'JACK US EQUITY'
analyst_upside = 98
analyst_downside = 73
analyst_pt_wic = 88
lookback = 120
unaffected_date = '2017-05-16'
target_date = "2019-02-21"
api_host = bbgclient.bbgclient.get_next_available_host()
fperiod = "1BF"

In [ ]:
adjustments_df_bear = None # '[{"PX":"0","BEST_EPS":"0","BEST_NET_INCOME":"0","BEST_OPP":"0","BEST_SALES":"0","CUR_EV_COMPONENT":"0","CUR_MKT_CAP":"0","DIVIDEND_INDICATED_YIELD":"0","BEST_CAPEX":"0","BEST_EBITDA":"0","EQY_SH_OUT":"0"}]'
adjustments_df_bull = None #'[{"PX":"0","BEST_EPS":"0","BEST_NET_INCOME":"0","BEST_OPP":"0","BEST_SALES":"0","CUR_EV_COMPONENT":"0","CUR_MKT_CAP":"0","DIVIDEND_INDICATED_YIELD":"0","BEST_CAPEX":"0","BEST_EBITDA":"0","EQY_SH_OUT":"0"}]'
adjustments_df_pt = None #'[{"PX":"0","BEST_EPS":"0","BEST_NET_INCOME":"0","BEST_OPP":"0","BEST_SALES":"0","CUR_EV_COMPONENT":"0","CUR_MKT_CAP":"0","DIVIDEND_INDICATED_YIELD":"0","BEST_CAPEX":"0","BEST_EBITDA":"0","EQY_SH_OUT":"0"}]'
bear_flag = None
bull_flag = None
pt_flag = None

In [3]:
OLS_regression_analysis_all_changes.bloomberg_peers("JACK US EQUITY")

['MCD US EQUITY',
 'WEN US EQUITY',
 'YUM US EQUITY',
 'LOCO US EQUITY',
 'YUMC US EQUITY',
 'QSR CN EQUITY']

In [4]:
#Run the below line for just the regression output -- can save this down, but must be passed throught the calculations function

In [5]:
results = OLS_regression_analysis_all_changes.run_regression_optimal_peers(alpha_ticker, unaffected_date, lookback_period = 120)

In [6]:
results

({'P/EPS': {'MCD US EQUITY': -0.6547756043537755,
   'WEN US EQUITY': 0.37462814956534457,
   'YUM US EQUITY': -0.34444053365822525,
   'LOCO US EQUITY': 0.33070785834113925,
   'YUMC US EQUITY': -0.08722756634491818,
   '(Intercept)': 27.2978193177026},
  'EV/EBITDA': {'MCD US EQUITY': -0.27923586403912704,
   'WEN US EQUITY': 0.1944470719612887,
   'LOCO US EQUITY': 0.5736259842543897,
   'YUMC US EQUITY': -0.17367176979797408,
   '(Intercept)': 8.970655769478123},
  'EV/Sales': {'WEN US EQUITY': 0.3342208652028911,
   'YUM US EQUITY': -0.21085478598188387,
   'QSR CN EQUITY': 0.19913429453699316,
   '(Intercept)': 0.7771808754094983},
  'DVD yield': {'MCD US EQUITY': -0.26342622295893103,
   'WEN US EQUITY': 0.5367546868081103,
   'YUM US EQUITY': -0.02244112149128218,
   '(Intercept)': 1.3078452678455268},
  'FCF yield': {'MCD US EQUITY': -0.3006376695581049,
   'WEN US EQUITY': 0.7022377820172085,
   'LOCO US EQUITY': 0.2074864539848666,
   'QSR CN EQUITY': -0.21844955409410574,
 

In [10]:
def run_OLS_regression_analysis(alpha_ticker, tgtDate, analyst_upside, analyst_downside, analyst_pt_wic,
                                regression_results, api_host, adjustments_df_bear=None, adjustments_df_bull=None,
                                adjustments_df_pt=None, bear_flag=None, bull_flag=None, pt_flag=None,
                                f_period = '1BF'):
    metrics = ['P/EPS', "EV/EBITDA", "EV/Sales", 'DVD yield', 'FCF yield']
    slicer = dfutils.df_slicer()
    price_tgt_dt = datetime.datetime.strptime(tgtDate, '%Y-%m-%d')
    as_of_dt = datetime.datetime.today()
    
    if len(regression_results) == 2:
        optimal_results = regression_results[0]
        overlap_results = regression_results[1]
    else:
        optimal_results = regression_results
        
    peers = []
    for metric in metrics:  
        results = optimal_results[metric]
        for x in optimal_results[metric]:
            if x not in peers:
                peers.append(x)
    peers.remove('(Intercept)')
    
    peer2ptd_multiple = {p:OLS_regression_analysis_all_changes.multiples_df(p,slicer.prev_n_business_days(
        100,price_tgt_dt).strftime('%Y%m%d'),price_tgt_dt.strftime('%Y%m%d'),api_host,fperiod='1BF',
                                                                    multiples_to_query=metrics) for p in peers}
    peer2now_multiple = {p:OLS_regression_analysis_all_changes.multiples_df(p,slicer.prev_n_business_days(
        100,as_of_dt).strftime('%Y%m%d'),as_of_dt.strftime('%Y%m%d'),api_host,fperiod='1BF',
                                                                    multiples_to_query=metrics) for p in peers}
    
    dataframes = {
        'peer2ptd_multiple': peer2ptd_multiple,
        'peer2now_multiple': peer2now_multiple
    }

    adjusted_results = OLS_regression_analysis_all_changes.premium_analysis_df_OLS2_quick_all_changes(dataframes, 
                                                                                              regression_results,
                                                                                              alpha_ticker, 
                                                                                              analyst_upside, 
                                                                                              analyst_downside,
                                                                                              analyst_pt_wic,
                                                                                              tgtDate, api_host, 
                                                                                              adjustments_df_bear=None,
                                                                                              adjustments_df_bull=None,
                                adjustments_df_pt=None, bear_flag=None, bull_flag=None, pt_flag=None, fperiod="1BF")
    
    return adjusted_results

In [11]:
x = run_OLS_regression_analysis(alpha_ticker, target_date, analyst_upside, analyst_downside, analyst_pt_wic, 
                            results, api_host, adjustments_df_bear=None, adjustments_df_bull=None,
                            adjustments_df_pt=None, bear_flag=None, bull_flag=None, pt_flag=None, f_period='1BF')

In [12]:
res = x['optimal_peer_results_df']
res

,Metric,Alpha Upside (analyst),Alpha Downside (analyst),Alpha PT WIC (analyst),Peers Multiples DataFrame @ Price Target Date,Peers Multiples @ Price Target Date,Alpha Implied Multiple @ Price Target Date,Alpha Balance Sheet DataFrame (Bear Case),Alpha Bear Multiple @ Price Target Date,Alpha Balance Sheet DataFrame (Bull Case),...,Premium Bull (%),Peers Multiples DataFrame @ Now,Peers Multiples @ Now,Alpha Implied Multiple @ Now,Alpha Bear Multiple @ Now,Alpha PT WIC Multiple @ Now,Alpha Bull Multiple @ Now,Alpha Downside,Alpha PT WIC,Alpha Upside
0,P/EPS,98,73,88,Peer Multiple 0 MCD US EQUITY ...,"{'MCD US EQUITY': 22.15, 'WEN US EQUITY': 27.1...",18.595748,Date PX CUR_MKT_CAP EQY_SH_OUT ...,16.504635,Date PX CUR_MKT_CAP EQY_SH_OUT ...,...,19.150392,Peer Multiple 0 MCD US EQUITY ...,"{'MCD US EQUITY': 24.088, 'WEN US EQUITY': 27....",16.117494,14.305063,17.244460,19.204057,63.271294,76.272245,84.939545
1,EV/EBITDA,98,73,88,Peer Multiple 0 MCD US EQUITY ...,"{'MCD US EQUITY': 15.787, 'WEN US EQUITY': 14....",11.486520,Date PX CUR_MKT_CAP EQY_SH_OUT ...,11.007356,Date PX CUR_MKT_CAP EQY_SH_OUT ...,...,16.888055,Peer Multiple 0 MCD US EQUITY ...,"{'MCD US EQUITY': 17.87, 'WEN US EQUITY': 17.2...",11.714574,11.225898,12.706122,13.692938,75.258580,90.556392,100.754933
2,EV/Sales,98,73,88,Peer Multiple 0 WEN US EQUITY ...,"{'WEN US EQUITY': 3.861, 'YUM US EQUITY': 7.07...",2.040302,Date PX CUR_MKT_CAP EQY_SH_OUT ...,3.067503,Date PX CUR_MKT_CAP EQY_SH_OUT ...,...,83.385941,Peer Multiple 0 WEN US EQUITY ...,"{'WEN US EQUITY': 4.462, 'YUM US EQUITY': 7.30...",2.241999,3.370745,3.815205,4.111511,84.245775,100.728620,111.717184
3,DVD yield,98,73,88,Peer Multiple 0 MCD US EQUITY ...,"{'MCD US EQUITY': 2.5382, 'WEN US EQUITY': 2.2...",1.807715,Date PX CUR_MKT_CAP EQY_SH_OUT ...,2.191790,Date PX CUR_MKT_CAP EQY_SH_OUT ...,...,-9.683805,Peer Multiple 0 MCD US EQUITY ...,"{'MCD US EQUITY': 2.3222, 'WEN US EQUITY': 2.1...",1.797878,2.179863,1.808295,1.623775,73.399410,88.481481,98.536194
4,FCF yield,98,73,88,Peer Multiple 0 MCD US EQUITY ...,"{'MCD US EQUITY': 0.036694521427253456, 'WEN U...",0.040904,Date PX CUR_MKT_CAP EQY_SH_OUT ...,0.167521,Date PX CUR_MKT_CAP EQY_SH_OUT ...,...,205.067426,Peer Multiple 0 MCD US EQUITY ...,"{'MCD US EQUITY': 0.03466913787178018, 'WEN US...",0.041818,0.171263,0.142071,0.127574,71.404660,86.076851,95.858311
